In [ ]:
def fun_asymp(data = True, max_dB = 12, modulation = 'QAM', order = 2, show_fig = True):
  # Ber and Ser asymptotes for M-PSK. By Adrian Ramirez
  # Relation between EsN0 and EbN0. Ref: https://www.mathworks.com/help/comm/ug/awgn-channel.html
  # To compare with Matlab bertool curves. Ref: https://www.mathworks.com/help/comm/ref/berawgn.html
  # SER for M-PSK: https://nl.mathworks.com/help/comm/ug/analytical-expressions-used-in-berawgn-function-and-bit-error-rate-analysis-app.html#responsive_offcanvas
  # Integration: #https://docs.scipy.org/doc/scipy/tutorial/integrate.html, https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad
  
  M = order # modulation order
  k = np.log2(M) # bits per symbols

  vct_dB = np.arange(max_dB)
  vct_ber_asymp = np.zeros(max_dB)
  vct_ser_asymp = np.zeros(max_dB)

  if show_fig == True:
    plt.figure(figsize=(14, 6.5), dpi= 80, facecolor='w', edgecolor='k')
    ax3 = plt.subplot(1,2,1)
    ax4 = plt.subplot(1,2,2)

  vct_hamming = np.zeros(M)
  for constellation_point in np.arange(0, M):
    point_bin = np.binary_repr(constellation_point, width=4)
    point_gray = binary_to_gray(point_bin)
    vct_hamming[constellation_point] = point_gray.count('1')

  for value_dB in vct_dB:
    value_x = 10**(value_dB/10)

    if modulation == 'QAM':

      # BER for M-QAM
      prod_1 = 2/(np.sqrt(M)*np.log2(np.sqrt(M)))
      prod_2 = 0
      for k in range(1, int( np.log2(np.sqrt(M))+1 )):
        for i in range(0, int( ((1-2**(-k))*np.sqrt(M)-1)+1 )):
          qu = norm.sf((2*i+1)*np.sqrt(6*np.log2(M)*value_x/(2*(M-1))))
          prod_2 += ((-1)**np.floor((i*2**(k-1))/np.sqrt(M)))*(2**(k-1)-np.floor((i*2**(k-1))/np.sqrt(M)+1/2))*qu
      vct_ber_asymp[value_dB] = prod_1*prod_2

      # SER for M-QAM
      factor_1 = ( 4*(np.sqrt(M)-1)/np.sqrt(M) )*( norm.sf( np.sqrt(3*value_x/(M-1)) )) # value_x = k*Eb/No
      factor_2 = ( 4*((np.sqrt(M)-1)/np.sqrt(M))**2 )*(( norm.sf( np.sqrt(3*value_x/(M-1)) ) )**2)
      vct_ser_asymp[value_dB] = factor_1-factor_2

    elif modulation == 'PSK':

      # BER for M-PSK
      P_b = 0
      for i in range(1, M//2+1):
        if i == M//2:
          w = vct_hamming[i]
        else:
          w = vct_hamming[i] + vct_hamming[M-i]
        integ_1 = integrate.quad(lambda theta: np.exp(-k*value_x*((np.sin((2*i-1)*PI/M))**2)/((np.sin(theta))**2) ), 0, PI*(1-(2*i-1)/M))
        integ_2 = integrate.quad(lambda theta: np.exp(-k*value_x*((np.sin((2*i+1)*PI/M))**2)/((np.sin(theta))**2) ), 0, PI*(1-(2*i+1)/M))
        P_i = (1/(2*PI))*(integ_1[0] - integ_2[0])
        P_b += (1/k)*w*P_i
      vct_ber_asymp[value_dB] = P_b

      # SER for M-PSK
      integration = integrate.quad(lambda theta: np.exp(-value_x*((np.sin(PI/M))**2)/((np.sin(theta))**2) ), 0, (M-1)*PI/M) # Considering EsNo = k*EbNo
      vct_ser_asymp[value_dB] = (1/PI)*integration[0]
      #print('integrate.quad error: ', integration[1])

    else:
      print('Invalid value at modulation key. Use literals QAM or PSK.')
      return None

    if show_fig == True and data == True:
      ax3.annotate('({}, {})'.format(value_dB, round(vct_ber_asymp[value_dB], 8)),(value_dB, vct_ber_asymp[value_dB]))   # annotate method puts data points in the graphic
      ax4.annotate('({}, {})'.format(value_dB, round(vct_ser_asymp[value_dB], 8)),(value_dB, vct_ser_asymp[value_dB]))   # other way: plt.annotate('text', (x,y))

  if show_fig == True:
    ax3.set_title('BER vs EbN0 in dB');
    ax3.semilogy(vct_dB, vct_ber_asymp, 'b.-')
    ax3.grid(True)
    ax4.set_title('SER vs EsN0 in dB');
    ax4.semilogy(vct_dB, vct_ser_asymp, 'b.-')
    ax4.grid(True)
  
  return (vct_ber_asymp, vct_ser_asymp)